In [8]:
import dspy
from dspy.datasets import HotPotQA
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate.evaluate import Evaluate
import os
os.environ["OLLAMA_API_BASE"] = "http://localhost:11434" 

In [19]:
# Configure DSPy with Ollama (Llama3)
lm = dspy.LM('ollama_chat/llama3', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

# Simple dataset - Animal facts
animal_facts = [
    "Elephants are the largest land mammals and can weigh up to 14,000 pounds.",
    "Dolphins are highly intelligent marine mammals that communicate using clicks and whistles.",
    "Penguins are flightless birds that live primarily in the Southern Hemisphere.",
    "Cheetahs are the fastest land animals, capable of reaching speeds of 70 mph.",
    "Octopuses have three hearts and blue blood.",
    "Giraffes have the same number of neck vertebrae as humans - seven.",
    "Sharks have been around for more than 400 million years.",
    "Bees communicate through a waggle dance to share information about food sources."
]

In [20]:
class GenerateAnswer(dspy.Signature):
    context = dspy.InputField(desc="relevant information")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="a short, accurate answer")


In [ ]:
class QASystem(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        # Simple retrieval - find relevant facts
        relevant_context = self.retrieve_relevant_facts(question)
        
        # Generate answer using the context
        prediction = self.generate_answer(
            context=relevant_context, 
            question=question
        )
        return dspy.Prediction(answer=prediction.answer)
    
    def retrieve_relevant_facts(self, question):
        question_lower = question.lower()
        relevant_facts = []
        
        for fact in animal_facts:
            # Check if any words in the question appear in the fact
            if any(word in fact.lower() for word in question_lower.split() if len(word) > 3):
                relevant_facts.append(fact)
        
        return " ".join(relevant_facts) if relevant_facts else "No specific information available."


In [22]:
# Initialize the QA system
qa_system = QASystem()

# Example questions and answers for testing
def test_qa_system():
    questions = [
        "How fast can a cheetah run?",
        "What is special about octopus blood?",
        "How do bees communicate?",
        "What is the largest land mammal?",
        "Do giraffes have more neck bones than humans?"
    ]
    
    print("=== Question-Answering Assistant ===\n")
    
    for question in questions:
        try:
            result = qa_system(question)
            print(f"Q: {question}")
            print(f"A: {result.answer}")
            print("-" * 50)
        except Exception as e:
            print(f"Error answering '{question}': {e}")
            print("-" * 50)

In [23]:
# Run the test
if __name__ == "__main__":
    test_qa_system()
    
    # Interactive mode
    print("\n=== Interactive Mode ===")
    print("Ask questions about animals (type 'quit' to exit):")
    
    while True:
        question = input("\nYour question: ").strip()
        if question.lower() in ['quit', 'exit', 'q']:
            break
        
        if question:
            try:
                result = qa_system(question)
                print(f"Answer: {result.answer}")
            except Exception as e:
                print(f"Error: {e}")
        else:
            print("Please enter a question.")

=== Question-Answering Assistant ===



C:\Users\zyad3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## co...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Q: How fast can a cheetah run?
A: Up to 70 mph
--------------------------------------------------
Q: What is special about octopus blood?
A: Octopus blood is blue due to its unique composition containing hemocyanin, which replaces the need for hemoglobin.
--------------------------------------------------
Q: How do bees communicate?
A: Bees communicate through a waggle dance, as well as chemical signals, body language, and vibrations.
--------------------------------------------------
Q: What is the largest land mammal?
A: Elephants
--------------------------------------------------
Q: Do giraffes have more neck bones than humans?
A: No
--------------------------------------------------

=== Interactive Mode ===
Ask questions about animals (type 'quit' to exit):
